In [1]:
import FinanceDataReader as fdr

In [24]:
import pandas as pd
import numpy as np

In [181]:
def get_bollinger_band_back_testing(symbol="005930", start_date="1990-01-01"):
    df = fdr.DataReader(symbol, start=start_date)
    current_price = df["Close"].iloc[-1]

    df["M20"] = df["Close"].rolling(20).mean()
    df["std"] = df["Close"].rolling(20).std()
    df["HighLine"] = df["M20"] + df["std"] * 2
    df["LowLine"] = df["M20"] - df["std"] * 2
    cond = df["Close"] <= df["LowLine"]
    close_cond = df.loc[cond, "Close"]
    
    buy_dict = dict()
    for date, close in close_cond.items():
        date = "_".join([str(date.year), str(date.month)]) # 2020_10 year_month
        if date not in buy_dict:
            buy_dict[date] = close
    date_list = list(buy_dict.keys())
    close_list = list(buy_dict.values())
    _start_date = start_date[:7].replace("-", "_")
    bought_dict = {"num": 0, "total_buy_price": 0}

    for idx in range(len(close_list)):
        date = date_list[idx]

        if idx == 0:
            before_date = _start_date
        else:
            before_date = date_list[idx-1]

        year = int(date.split("_")[0])
        month = int(date.split("_")[1])
        before_year = int(before_date.split("_")[0])
        before_month = int(before_date.split("_")[1])

        dif_year = year - before_year
        dif_month = month - before_month

        dif_month = dif_year * 12 + dif_month

        bought_dict["num"] += dif_month
        bought_dict["total_buy_price"] += dif_month*close_list[idx]
    bought_dict["AP"] = int(bought_dict["total_buy_price"] / bought_dict["num"])
    bought_dict["CP"] = current_price
    bought_dict["profit_rate"] = round((bought_dict["CP"]-bought_dict["AP"])/bought_dict["AP"]*100, 2)
    bought_dict["profit"] = int(bought_dict["total_buy_price"]*bought_dict["profit_rate"]/100)
    
    for key, value in bought_dict.items():
        suffix = ""
        prefix = ""
        
        if key in ["total_buy_price", "AP", "CP", "profit"]:
            suffix = "원"
        
        elif key in ["num"]:
            suffix = "주"
        
        elif key in ["profit_rate"]:
            suffix = "%"
        
        if key in ["profit_rate", "profit"]:
            if value > 0:
                prefix = "+"
        
        value = prefix+format(value, ",")+suffix
        bought_dict[key] = value
    return bought_dict

In [182]:
get_bollinger_band_back_testing("079160", "2019-01-01")

{'num': '21주',
 'total_buy_price': '577,181원',
 'AP': '27,484원',
 'CP': '21,500원',
 'profit_rate': '-21.77%',
 'profit': '-125,652원'}